In [25]:
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from IPython.display import clear_output
from datetime import datetime
import requests

In [2]:
# Input path ke chromedriver.exe
path_chromedriver = os.path.join(os.getcwd(), "chromedriver-win64", "chromedriver.exe")

chrome_options = Options()
service = Service(executable_path=path_chromedriver)

In [66]:
list_daerah = [
"jakarta",
"banten",
"jawa-barat",
"jawa-tengah-diy",
"jawa-timur",
"kalimantan",
"sulawesi",
"bali-nusatenggara",
"maluku-papua",
"sumbagsel",
"sumbagteng",
"sumbagut"
]

In [135]:
# Loop melalui semua peserta
def data_district(district):
    participants = driver.find_elements(By.CLASS_NAME, "champion-card")
    district = district

    for champion in participants:
            # Ambil data dari setiap champion-card
            name = champion.find_element(By.CLASS_NAME, "champion-card-name").text
            school = champion.find_element(By.CLASS_NAME, "champion-card-school").text
            gpa = champion.find_element(By.CLASS_NAME, "champion-card-score").text
            image_url = champion.find_element(By.CLASS_NAME, "profile-champion-image").get_attribute("src")
            username = champion.find_element(By.CLASS_NAME, "sosmed-label").text
            awards = [award.text for award in champion.find_elements(By.CLASS_NAME, "honorable-detail")]
            image_url = champion.find_element(By.CLASS_NAME, "profile-champion-image").get_attribute("src")
            district_name = driver.find_element(By.CLASS_NAME, "district-city-label").text

            data_list.append({
                 "District": district,
                 "District Name": district_name,
                "Name": name,
                "School": school,
                "GPA": gpa,
                "Username": username,
                "Awards": awards
            })

            # Download gambar
            image_path = f"images aoc/Champion/D{district} {name}.png"
            response = requests.get(image_url, stream=True)
            if response.status_code == 200:
                with open(image_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)

            # Locate the image element using its class name
            dis_image_element = driver.find_element(By.CLASS_NAME,'district-logo')

            # Get the image URL from the 'src' attribute
            dis_image_url = dis_image_element.get_attribute('src')

            # Use requests to download the image
            dis_img_data = requests.get(dis_image_url).content

            # Save the image to a file
            with open(f"images aoc/Logo/D{district}.png", "wb") as file:
                file.write(dis_img_data)

            # Locate the image element by its class
            main_image_element = driver.find_element(By.CLASS_NAME, 'hero-image.desktop')

            # Extract the image URL
            main_image_url = main_image_element.get_attribute('src')

            # Use requests to download the image
            main_img_data = requests.get(main_image_url).content

            # Save the image to a local file
            with open(f"images aoc/Main/D{district} Main Image.png", "wb") as file:
                file.write(main_img_data)

In [136]:
def coach_data(district):
            coach_name = driver.find_element(By.CLASS_NAME,'coach-card-name').text
            coach_school = driver.find_element(By.CLASS_NAME,'coach-card-school').text
            social_media = driver.find_element(By.CLASS_NAME,'sosmed-label').text
            image_url = driver.find_element(By.CLASS_NAME,'profile-coach-image').get_attribute('src')

            # Honorable mentions - Collect all the honorable mentions listed
            honorable_mentions = []
            # Now, find the honorable mentions container specifically for the coach
            honorable_mentions_container = driver.find_element(By.CLASS_NAME, 'honorable-mentions-container.coach')

            # Extract all honorable mentions for the coach
            honorable_mentions = []
            honorable_details = honorable_mentions_container.find_elements(By.CLASS_NAME, 'honorable-detail')
            for mention in honorable_details:
                honorable_mentions.append(mention.text)
        
            # Download and save the image
            img_data = requests.get(image_url).content
            with open(f'images aoc/Coach/ D{district} {coach_name}.png', 'wb') as files:
                files.write(img_data)
            
            coach_dataset.append({
                "District": district,
                "District Name": driver.find_element(By.CLASS_NAME, "district-city-label").text,
                "Coach Name": coach_name,
                "Coach School": coach_school,
                "Social Media": social_media,
                "Honorable Mentions": honorable_mentions
            })

In [98]:
# Open Chrome Browser
driver = webdriver.Chrome(service=service, options=chrome_options)


In [140]:
coach_dataset = []
data_list = []

In [141]:
for index,daerah in enumerate(list_daerah,start=1):
    url = f"https://www.ruangguru.com/champions/aoc/{daerah}"
    driver.get(url)
    data_district(index)
    coach_data(index)

In [131]:
coach_dataset = pd.DataFrame(coach_dataset)
coach_dataset.to_excel("Coach Data.xlsx", index=False)

data_list = pd.DataFrame(data_list)
data_list.to_excel("Champion Peserta.xlsx", index=False)